Importing Liberties  

In [1]:
import yfinance as yf
import pandas as pd  
import requests                                                                     # Importing packages from my Python venv 
from bs4 import BeautifulSoup 
import sqlite3
import locale

Data Scraping 

In [2]:
sp500_url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'              # Assigning link to 'sp500_url'
response = requests.get(sp500_url)                                                  # Using request to get access to 'sp500_url'

In [3]:
if response.status_code ==200:
    print('Request successful')
else:                                                                              # Checking status = 200 code using 'if' & 'else'
    print('Request not successful')

Request successful


In [4]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find ('table')                                       # Using Beautifulsoup() to find table within my url in 'response'

In [5]:
data_table = pd.read_html(str(table))[0]            # Using pandas to read link from table and assigning a value to a variable 'data_table'

/var/folders/80/lqqh37s14jn1pbg0fb61rf5c0000gp/T/ipykernel_90277/714230589.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_table = pd.read_html(str(table))[0]            # Using pandas to read link from table and assigning a value to a variable 'data_table'


In [6]:
print("Resulting DataFrame:")

print(data_table.head(503))     

Resulting DataFrame:
    Symbol              Security             GICS Sector  \
0      MMM                    3M             Industrials   
1      AOS           A. O. Smith             Industrials   
2      ABT                Abbott             Health Care   
3     ABBV                AbbVie             Health Care   
4      ACN             Accenture  Information Technology   
..     ...                   ...                     ...   
498    YUM           Yum! Brands  Consumer Discretionary   
499   ZBRA    Zebra Technologies  Information Technology   
500    ZBH         Zimmer Biomet             Health Care   
501   ZION  Zions Bancorporation              Financials   
502    ZTS                Zoetis             Health Care   

                      GICS Sub-Industry    Headquarters Location  Date added  \
0              Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1                     Building Products     Milwaukee, Wisconsin  2017-07-26   
2                 

Data Formatting .tolist()

In [7]:
tickers = data_table ['Symbol'].tolist()                                                              # Assigning 'tickers' to only the 'Symbol' of the 'data_table' and convert into a list()
tickers_list = [ticker.replace('BF.B', 'BF-B').replace('BRK.B', 'BRK-B') for ticker in tickers]       # Replacing two incorrect Value 'tickers' within a variable 'tickers_list'

print('Tickers list')
print(tickers_list) 

Tickers list
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CDAY', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CS

In [8]:
sectors_list = data_table ['GICS Sector'].tolist()       # Convert 'GICS Sector' into a list() within 'data_table' assigned to 'sectors_list'

print('Sectors list')
print(sectors_list)

Sectors list
['Industrials', 'Industrials', 'Health Care', 'Health Care', 'Information Technology', 'Information Technology', 'Information Technology', 'Utilities', 'Financials', 'Health Care', 'Materials', 'Consumer Discretionary', 'Information Technology', 'Materials', 'Real Estate', 'Health Care', 'Industrials', 'Utilities', 'Financials', 'Communication Services', 'Communication Services', 'Consumer Staples', 'Consumer Discretionary', 'Materials', 'Utilities', 'Industrials', 'Utilities', 'Financials', 'Financials', 'Real Estate', 'Utilities', 'Financials', 'Industrials', 'Health Care', 'Information Technology', 'Information Technology', 'Information Technology', 'Financials', 'Energy', 'Information Technology', 'Information Technology', 'Consumer Discretionary', 'Financials', 'Consumer Staples', 'Information Technology', 'Financials', 'Financials', 'Communication Services', 'Utilities', 'Information Technology', 'Industrials', 'Consumer Discretionary', 'Real Estate', 'Materials', 'I

In [9]:
company_list = data_table ['Security'].tolist()         # Convert 'Security' into a list() within 'data_table' assigned to 'stocks_list'

print('Stocks list')
print(company_list)

Stocks list
['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automated Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corpora

In [10]:
Industry_list = data_table ['GICS Sub-Industry'].tolist()                     # Convert 'GICS Sub-Industry' into a list() within 'data_table' assigned to 'Industry_list'

print('Industry list')
print(Industry_list)

Industry list
['Industrial Conglomerates', 'Building Products', 'Health Care Equipment', 'Biotechnology', 'IT Consulting & Other Services', 'Application Software', 'Semiconductors', 'Independent Power Producers & Energy Traders', 'Life & Health Insurance', 'Life Sciences Tools & Services', 'Industrial Gases', 'Hotels, Resorts & Cruise Lines', 'Internet Services & Infrastructure', 'Specialty Chemicals', 'Office REITs', 'Health Care Supplies', 'Building Products', 'Electric Utilities', 'Property & Casualty Insurance', 'Interactive Media & Services', 'Interactive Media & Services', 'Tobacco', 'Broadline Retail', 'Paper & Plastic Packaging Products & Materials', 'Multi-Utilities', 'Passenger Airlines', 'Electric Utilities', 'Consumer Finance', 'Multi-line Insurance', 'Telecom Tower REITs', 'Water Utilities', 'Asset Management & Custody Banks', 'Electrical Components & Equipment', 'Biotechnology', 'Electronic Components', 'Semiconductors', 'Application Software', 'Insurance Brokers', 'Oil &

In [11]:
Location_list = data_table ['Headquarters Location'].tolist()                                  # Convert 'Headquarters Location' into a list() within 'data_table' assigned to 'Location_list'

print('Headquarters Location list')
print(Location_list)

Headquarters Location list
['Saint Paul, Minnesota', 'Milwaukee, Wisconsin', 'North Chicago, Illinois', 'North Chicago, Illinois', 'Dublin, Ireland', 'San Jose, California', 'Santa Clara, California', 'Arlington, Virginia', 'Columbus, Georgia', 'Santa Clara, California', 'Allentown, Pennsylvania', 'San Francisco, California', 'Cambridge, Massachusetts', 'Charlotte, North Carolina', 'Pasadena, California', 'Tempe, Arizona', 'Dublin, Ireland', 'Madison, Wisconsin', 'Northbrook, Illinois', 'Mountain View, California', 'Mountain View, California', 'Richmond, Virginia', 'Seattle, Washington', 'Warmley, Bristol, United Kingdom', 'St. Louis, Missouri', 'Fort Worth, Texas', 'Columbus, Ohio', 'New York City, New York', 'New York City, New York', 'Boston, Massachusetts', 'Camden, New Jersey', 'Minneapolis, Minnesota', 'Berwyn, Pennsylvania', 'Thousand Oaks, California', 'Wallingford, Connecticut', 'Wilmington, Massachusetts', 'Canonsburg, Pennsylvania', 'London, UK', 'Houston, Texas', 'Cupertino

DataFrame Creation

In [12]:

Sp500_columns = pd.DataFrame({
    'Ticker': tickers_list,
    'Company': company_list,                                                  # Adding my variables ('tickers_list','stocks_list, and 'sectors_list') to a new Dataframe 'Sp500_columns' using pandas & {}
    'Sector': sectors_list,
    'Industry': Industry_list,
    'Location': Location_list,}) 

print(Sp500_columns)

    Ticker               Company                  Sector  \
0      MMM                    3M             Industrials   
1      AOS           A. O. Smith             Industrials   
2      ABT                Abbott             Health Care   
3     ABBV                AbbVie             Health Care   
4      ACN             Accenture  Information Technology   
..     ...                   ...                     ...   
498    YUM           Yum! Brands  Consumer Discretionary   
499   ZBRA    Zebra Technologies  Information Technology   
500    ZBH         Zimmer Biomet             Health Care   
501   ZION  Zions Bancorporation              Financials   
502    ZTS                Zoetis             Health Care   

                               Industry                 Location  
0              Industrial Conglomerates    Saint Paul, Minnesota  
1                     Building Products     Milwaukee, Wisconsin  
2                 Health Care Equipment  North Chicago, Illinois  
3          

Importing Data From Library

In [13]:
tickers_volume = yf.download(tickers_list, period="1y", interval="1d")[['Volume']]    # Using 'yf' to download all the info in the 'tickers_list' between those dates and assigning it to 'tickers_price'

[*********************100%%**********************]  503 of 503 completed


Calculate The Volume For Each Ticker

In [14]:
daily_volume = tickers_volume['Volume'].resample('D').sum()                                                # Calculate the daily volume for each ticker

monthly_volume = tickers_volume['Volume'].resample('M').mean()                                           # Calculate the monthly average volume for each ticker

weekly_average_volume = tickers_volume['Volume'].resample('W').mean()                               # Calculate the daily average volume for each ticker

yearly_average_volume = tickers_volume['Volume'].resample('Y').mean()                               # Calculate the yearly average volume for each ticker

Importing Market Caps From Each Ticker 

In [15]:
data = []

for ticker in tickers_list:
    try:                                             # Importing Tickers from S&P 500 (thicker_list)              
        stock = yf.Ticker(ticker)
        info = stock.info
        
        history = stock.history(period="W", interval="1d")
                                                     # Extracting information
        market_cap = info.get('marketCap', 'N/A')
        name =  'N/A'  
        sector = 'N/A'  
        
                                                     # Append the data to the list
        data.append({
            'Ticker': ticker,
            'Company': name,
            'Sector': sector,
            'Market_Cap': market_cap
        })

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")

                                                    # Create a DataFrame from the list of data
SP500 = pd.DataFrame(data)

SP500_MCS = SP500.sort_values(by='Market_Cap', ascending=False)


In [16]:
SP500_MCS.drop('Company', axis=1, inplace=True)
                                                                     # Dropping Unnecessary columns from DataFrame(SP500_MC)
SP500_MCS.drop('Sector', axis=1, inplace=True)

print(SP500_MCS)

    Ticker     Market_Cap
39    AAPL  3072766771200
318   MSFT  2755361767424
19   GOOGL  1667179872256
20    GOOG  1666174156800
22    AMZN  1549789954048
..     ...            ...
144   XRAY     7283746816
225    HAS     6990930432
321    MHK     6714014208
491    WHR     6530249728
501   ZION     6494851584

[503 rows x 2 columns]


Data Formatting (Market Cap) & Function Creation 

In [17]:
def format_market_cap(market_cap):
    if isinstance(market_cap, (int, float)):
                                                                   # If it's already a numeric value, format it accordingly
        if market_cap >= 1e12:
            return "${:.2f} T".format(market_cap / 1e12)
        elif market_cap >= 1e9:
            return "${:.2f} B".format(market_cap / 1e9)
        elif market_cap >= 1e6:
            return "${:.2f} M".format(market_cap / 1e6)
        else:
            return "${:.2f}".format(market_cap)
    else:
        try:
                                                                   # Convert the market cap value to a floating-point number
            market_cap = float(market_cap.replace('$', '').replace('B', 'e9').replace('M', 'e6').replace('T', 'e12'))

                                                                   # Check the magnitude of the market cap and append 'T', 'B', or 'M' accordingly
            if market_cap >= 1e12:
                return "${:.2f} T".format(market_cap / 1e12)
            elif market_cap >= 1e9:
                return "${:.2f} B".format(market_cap / 1e9)
            elif market_cap >= 1e6:
                return "${:.2f} M".format(market_cap / 1e6)
            else:
                return "${:.2f}".format(market_cap)
        except Exception as e:
            
            print(f"Error formatting market cap: {e}")
            return market_cap                                       # Return the original value if there's an issue with formatting


SP500_MCS['Market_Cap'] = SP500_MCS['Market_Cap'].apply(format_market_cap)

print(SP500_MCS)


    Ticker Market_Cap
39    AAPL    $3.07 T
318   MSFT    $2.76 T
19   GOOGL    $1.67 T
20    GOOG    $1.67 T
22    AMZN    $1.55 T
..     ...        ...
144   XRAY    $7.28 B
225    HAS    $6.99 B
321    MHK    $6.71 B
491    WHR    $6.53 B
501   ZION    $6.49 B

[503 rows x 2 columns]


Data Merging (JOIN) Drop And Inserting Column

In [18]:
Updated_Sp500_columns = pd.merge(Sp500_columns, SP500_MCS[['Ticker', 'Market_Cap']], left_on='Ticker', right_on='Ticker', how='right')             # pd.merge is used to merge two DataFrames 'JOIN'

                                                                                   
Updated_Sp500_columns.drop('Industry', axis=1, inplace=True)                                                                                        # Dropping Unnecessary Columns in merged DataFrames


Updated_Sp500_columns.insert(0, 'Rank', range(1, len(Updated_Sp500_columns) + 1))

 Function Creation 

In [19]:
def calculate_Date_weekly(ticker):
    OC_Date = yf.download(ticker, period="1y", interval="1d")
    
                                                             # Check if the index is a DatetimeIndex
    if not isinstance(OC_Date.index, pd.DatetimeIndex):
        raise KeyError(f"DatetimeIndex not found in {ticker} data.")

                                                             # Extract the date information from the index and resample it weekly, then calculate the mean
    Weekly_Date = OC_Date.index.to_frame().resample('W').mean()
    
                                                             # Get the most recent weekly date value
    most_recent_weekly_date = Weekly_Date.index[-1]
    
                                                             # Extract the date part from the string
    date_part = str(most_recent_weekly_date).split(' ')[0]

    return date_part

Updated_Sp500_columns['Date'] = Updated_Sp500_columns['Ticker'].apply(calculate_Date_weekly)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [20]:

def calculate_open_weekly_average(ticker):                                                                               # Defines a function to calculate the Open
    Open_Close = yf.download(ticker , period= "1y", interval="1d")
    Open_SP500 = Open_Close['Open'].resample('W').mean()                                                                 # Calculates the mean of the Open for each ticker
                                                                                                                         
    most_recent_weekly_open = Open_SP500.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_weekly_open 

Updated_Sp500_columns['Weekly_Open'] = Updated_Sp500_columns['Ticker'].apply(calculate_open_weekly_average)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [21]:
def calculate_close_weekly_average(ticker):                                                                               # Defines a function to calculate the Close
    Open_Close = yf.download(ticker, period= "1y", interval="1d")
    Close_SP500 = Open_Close['Close'].resample('W').mean()                                                # Calculates the mean of the Close for each ticker
    
    most_recent_weekly_close = Close_SP500.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_weekly_close  

Updated_Sp500_columns['Weekly_Close'] = Updated_Sp500_columns['Ticker'].apply(calculate_close_weekly_average)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Column Creation (Open/Close)

In [34]:
Updated_Sp500_columns['Weekly_Difference'] = (Updated_Sp500_columns['Weekly_Close'] - Updated_Sp500_columns['Weekly_Open']).astype(int)

In [23]:
def calculate_weekly_average(ticker):                                                                               # Defines a function to calculate the weekly average
    ticker_data = yf.download(ticker, period="1y", interval="1d")
    weekly_average = ticker_data['Volume'].resample('W').mean()                                                     # Calculates the mean of the weekly average volumes for each ticker
    
    most_recent_weekly_volume = weekly_average.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_weekly_volume  
  

Updated_Sp500_columns['Weekly_Volume'] = Updated_Sp500_columns['Ticker'].apply(calculate_weekly_average)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [24]:
def calculate_monthly_average(ticker):                                                                               # Defines a function to calculate the Daily Volume
    ticker_monthly_data = yf.download(ticker, period="1y", interval="1d")
    monthly_average = ticker_monthly_data['Volume'].resample('M').mean()                                                # Calculates the mean of the Daily Volume for each ticker
    
    most_recent_monthly_volume = monthly_average.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_monthly_volume    

Updated_Sp500_columns['Monthly_Volume'] = Updated_Sp500_columns['Ticker'].apply(calculate_monthly_average)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Data Formatting (Volume & Open/Close) 

In [25]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Average Weekly Open' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Weekly_Open'] = Updated_Sp500_columns['Weekly_Open'].apply(lambda x: locale.format_string("%d", x, grouping=True))

In [26]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Average Weekly Close' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Weekly_Close'] = Updated_Sp500_columns['Weekly_Close'].apply(lambda x: locale.format_string("%d", x, grouping=True))

In [27]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Weekly Average Volume' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Weekly_Volume'] = Updated_Sp500_columns['Weekly_Volume'].apply(lambda x: locale.format_string("%d", x, grouping=True))

In [28]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Daily Volume' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Monthly_Volume'] = Updated_Sp500_columns['Monthly_Volume'].apply(lambda x: locale.format_string("%d", x, grouping=True))

Changing Data Types

In [29]:
Updated_Sp500_columns['Weekly_Open'] = Updated_Sp500_columns['Weekly_Open'].astype(int)                   

Updated_Sp500_columns['Weekly_Close'] = Updated_Sp500_columns['Weekly_Close'].astype(int)                    # Converting Columns To int()
 
Updated_Sp500_columns['Weekly_Volume'] = Updated_Sp500_columns['Weekly_Volume'].astype(int)                   

Updated_Sp500_columns['Monthly_Volume'] = Updated_Sp500_columns['Monthly_Volume'].astype(int)               

Replacing Column Order 'POP'

In [35]:
Market_Cap = Updated_Sp500_columns.pop('Market_Cap')
Updated_Sp500_columns['Market_Cap'] = Market_Cap

print("Resulting S&P 500 DataFrame:")

Updated_Sp500_columns

Resulting S&P 500 DataFrame:


,Rank,Ticker,Company,Sector,Location,Date,Weekly_Open,Weekly_Close,Weekly_Difference,Weekly_Volume,Monthly_Volume,Market_Cap
0,1,AAPL,Apple Inc.,Information Technology,"Cupertino, California",2023-12-17,195,196,1,75826620,61525000,$3.07 T
1,2,MSFT,Microsoft,Information Technology,"Redmond, Washington",2023-12-17,371,371,0,41051660,32532072,$2.76 T
2,3,GOOGL,Alphabet Inc. (Class A),Communication Services,"Mountain View, California",2023-12-17,132,132,0,35962640,35261481,$1.67 T
3,4,GOOG,Alphabet Inc. (Class C),Communication Services,"Mountain View, California",2023-12-17,133,133,0,32937980,28188309,$1.67 T
4,5,AMZN,Amazon,Consumer Discretionary,"Seattle, Washington",2023-12-17,147,147,0,63411540,53271700,$1.55 T
...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,XRAY,Dentsply Sirona,Health Care,"Charlotte, North Carolina",2023-12-17,32,33,1,3515120,3302390,$7.28 B
499,500,HAS,Hasbro,Consumer Discretionary,"Pawtucket, Rhode Island",2023-12-17,49,49,0,2901780,2271345,$6.99 B
500,501,MHK,Mohawk Industries,Consumer Discretionary,"Calhoun, Georgia",2023-12-17,96,98,2,1496120,1186800,$6.71 B
501,502,WHR,Whirlpool Corporation,Consumer Discretionary,"Benton Harbor, Michigan",2023-12-17,113,115,2,1343480,1152481,$6.53 B


Checking Data Types

In [36]:
Updated_Sp500_columns.info()                              # Calling All The Info() Of This DataFrame 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Rank               503 non-null    int64 
 1   Ticker             503 non-null    object
 2   Company            503 non-null    object
 3   Sector             503 non-null    object
 4   Location           503 non-null    object
 5   Date               503 non-null    object
 6   Weekly_Open        503 non-null    int64 
 7   Weekly_Close       503 non-null    int64 
 8   Weekly_Difference  503 non-null    int64 
 9   Weekly_Volume      503 non-null    int64 
 10  Monthly_Volume     503 non-null    int64 
 11  Market_Cap         503 non-null    object
dtypes: int64(6), object(6)
memory usage: 47.3+ KB


Exporting DataFrame To SQLite With Primary Key (Recreation Of Tables)  

In [32]:
conn = sqlite3.connect('SP500_Database.db')

Updated_Sp500_columns.to_sql('SP500_Columns_Key', conn, index=False, if_exists='replace')   # Adding DataFrame To SQLite Database

cursor = conn.cursor()    # Starting SQL Query 

                          # Creating New Table With Primary Key
cursor.execute('''         
    CREATE TABLE IF NOT EXISTS SP500_Columns_Key_New (
        id INTEGER PRIMARY KEY,
        Rank INTEGER,
        Ticker TEXT,
        Company TEXT,
        Sector TEXT,
        Location TEXT,
        Date DATE,
        Weekly_Open INTEGER,
        Weekly_Close INTEGER,
        Weekly_Difference INTEGER,
        Weekly_Volume INTEGER,
        Monthly_Volume INTEGER,
        Market_Cap TEXT
    )''')


conn.commit()

                          # Copying Data From Existing Table To The New Table With Primary Key
cursor.execute('''
    INSERT INTO SP500_Columns_Key_New (Rank, Ticker, Company, Sector, Location, Date, Weekly_Open, Weekly_Close, Weekly_Difference, Weekly_Volume, Monthly_Volume, Market_Cap)    
    SELECT Rank, Ticker, Company, Sector, Location, Date, Weekly_Open, Weekly_Close, Weekly_Difference, Weekly_Volume, Monthly_Volume, Market_Cap
    FROM SP500_Columns_Key
''')

conn.commit()

                         # Renaming Both Tables One With Primary Key and One Without
cursor.execute('''
    ALTER TABLE SP500_Columns_Key RENAME TO SP500_Columns_Key_Old
''')

cursor.execute('''
    ALTER TABLE SP500_Columns_Key_New RENAME TO SP500_Columns_Key
''')

conn.commit()

                        # Dropping Old Table 
cursor.execute('''
    DROP TABLE IF EXISTS SP500_Columns_Key_Old
''')

conn.close()

Exporting DataFrame to Excel

In [33]:
Updated_Sp500_columns.to_excel('SP500_excel.xlsx', index=False) 